# Quantifying a Stellar Sample in the DP0 Catalogs

**S. Mau**, **A. Drlica-Wagner**

Adapted from "Looking at the "Object Catalogs": merged tract-patch catalogs in DC2 Run 1.1p" by **Michael Wood-Vasey** ([@wmwv](https://github.com/LSSTDESC/DC2-analysis/issues/new?body=@wmwv))

In [1]:
import os

import numpy as np
from scipy import stats

import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord

import healpy as hp

import fitsio as fits

In [2]:
figwidth = (7.1 - 0.3125) / 2  # 3.39375
figheight = 4.8 / 6.4 * figwidth  # 2.5453124999999996
figsize = (figwidth, figheight)
print(figsize)

(3.39375, 2.5453124999999996)


In [3]:
subplotpars = mpl.gridspec.SubplotParams(
    left=0.2,
    right=0.95,
    bottom=0.2,
    top=0.95,
    wspace=0.3,
    hspace=0.2
)

In [4]:
_original_bottom = subplotpars.bottom * figheight
_original_top = subplotpars.top * figheight

In [5]:
_new_top = figheight - _original_bottom
new_figheight = _original_top / _new_top * figheight
new_bottom = _original_bottom / new_figheight
new_top = 1 - new_bottom

In [6]:
print(new_top, new_bottom)

0.831578947368421 0.168421052631579


In [7]:
new_figsize = (figwidth, new_figheight)
new_subplotpars = mpl.gridspec.SubplotParams(
    left=0.2,
    right=0.95,
    bottom=new_bottom,
    top=new_top,
    wspace=0.3,
    hspace=0.2
)

# Query Data

In [8]:
# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

# Get an instance of the TAP service
service = get_tap_service("tap")
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"


In [9]:
# Define our reference position on the sky and cone radius in arcseconds
# to use in all following examples
coord = SkyCoord(ra=62.0*u.degree, dec=-37.0*u.degree, frame='icrs')
radius = 1 * u.deg

In [10]:
# tract = 3831  # contains (62, -37)

In [11]:
matched_filename = "obj_matched.csv"

In [12]:
if os.path.exists(matched_filename):
    df = pd.read_csv(matched_filename)
else:
    query = (
        f"SELECT mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec, ts.truth_type, "
        f"obj.objectId, obj.coord_ra, obj.coord_dec, "
        f"scisql_nanojanskyToAbMag(obj.g_cModelFlux) AS mag_g, "
        f"scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.g_cModelFluxErr) AS mag_err_g, "
        f"scisql_nanojanskyToAbMag(obj.r_cModelFlux) AS mag_r, "
        f"scisql_nanojanskyToAbMagSigma(obj.r_cModelFlux, obj.r_cModelFluxErr) AS mag_err_r, "
        f"scisql_nanojanskyToAbMag(obj.i_cModelFlux) AS mag_i, "
        f"scisql_nanojanskyToAbMagSigma(obj.i_cModelFlux, obj.i_cModelFluxErr) AS mag_err_i, "
        f"obj.refExtendedness as extendedness, "
        f"scisql_nanojanskyToAbMag(ts.flux_g) as truth_mag_g, "
        f"scisql_nanojanskyToAbMag(ts.flux_r) as truth_mag_r, "
        f"scisql_nanojanskyToAbMag(ts.flux_i) as truth_mag_i "
        f"FROM dp02_dc2_catalogs.MatchesTruth AS mt "
        f"JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "
        f"JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId "
        f"WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), CIRCLE('ICRS', {coord.ra.value}, {coord.dec.value}, {radius.value})) = 1 "
        # f"AND obj.tract = {tract} "
        # f"AND obj.detect_isPrimary = 1"
    )
    job = service.submit_job(query)

    job.run()

    job.wait(phases=['COMPLETED', 'ERROR'])
    print('Job phase is', job.phase)

    df = job.fetch_result().to_table().to_pandas()

    df.to_csv(matched_filename)

In [13]:
truth_filename = "truth.csv"

In [14]:
if os.path.exists(truth_filename):
    df_truth = pd.read_csv(truth_filename)
else:
    query_truth = (
        f"SELECT ts.id, ts.ra, ts.dec, ts.truth_type, "
        f"scisql_nanojanskyToAbMag(ts.flux_g) as truth_mag_g, "
        f"scisql_nanojanskyToAbMag(ts.flux_r) as truth_mag_r, "
        f"scisql_nanojanskyToAbMag(ts.flux_i) as truth_mag_i "
        f"FROM dp02_dc2_catalogs.TruthSummary AS ts "
        f"WHERE CONTAINS(POINT('ICRS', ts.ra, ts.dec), CIRCLE('ICRS', {coord.ra.value}, {coord.dec.value}, {radius.value})) = 1 "
    )
    job = service.submit_job(query_truth)

    job.run()

    job.wait(phases=['COMPLETED', 'ERROR'])
    print('Job phase is', job.phase)

    df_truth = job.fetch_result().to_table().to_pandas()

    df_truth.to_csv(truth_filename)

/tmp/ipykernel_636/1397482113.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_truth = pd.read_csv(truth_filename)


In [15]:
print(len(df), "objects")

1569034 objects


In [16]:
print(len(df_truth), "truth objects")

6877451 truth objects


## Paramaterization of completeness function and magnitute uncertainty

### Select good stars

In [17]:
# Matched objects passing S/N threshold
snr_threshold = 5
mag_err_threshold = 1 / snr_threshold
good_snr = df[
    (df['mag_err_g'] < mag_err_threshold)
    & (df['mag_err_r'] < mag_err_threshold)
]

In [18]:
# Matched objects with truth class of stars
true_stars = df[df['truth_type'] == 2]

In [19]:
# Matched objects classified as stars (and galaxies)
safe_max_extended = 1.0
stars = good_snr[good_snr['extendedness'] < safe_max_extended]
galaxies = good_snr[good_snr['extendedness'] >= safe_max_extended]

In [20]:
# Matched objects correctly classified as stars
# match_stars_class = stars[np.isin(stars['objectId'], true_stars['objectId'])]
match_stars_class = stars[np.in1d(stars['objectId'], true_stars['objectId'])]


# True stars
truth_stars = df_truth[df_truth['truth_type'] == 2]

In [21]:
# print("mag, stellar classification eff, stellar detection eff, detected & classified eff")
# out = np.array([centers,
#                 nstar_class/ntot_true_star,
#                 ntot_true_star/ntot_truth,
#                 nstar_class/ntot_truth]).T
# print(out)
# np.savetxt('eff.csv', out, delimiter=',', header='mag,classifiction_eff,detection_eff,classification_detection_eff')

# Efficieny Plot

In [22]:
efficiencies = pd.read_csv("eff.csv")

fig, axs = plt.subplots(
    1, 1,
    figsize=figsize,
    subplotpars=subplotpars,
)

axs.plot(
    efficiencies["# mag"],
    efficiencies["detection_eff"],
    label='Detection',
    ls=":",
    c="k",
)  # matched objects with truth class of stars / true stars
axs.plot(
    efficiencies["# mag"],
    efficiencies["classifiction_eff"],
    label='Classification',
    ls="--",
    c="k",
)  # matched objects correctly classified as stars / matched objects with truth class of stars
axs.plot(
    efficiencies["# mag"],
    efficiencies["classification_detection_eff"],
    label='Detection &\nClassification',
    ls="-",
    c="r",
)  # matched objects correctly classified as stars / true stars
axs.legend(loc="lower left")

axs.set_xlabel("$r$ [mag]")
axs.set_ylabel("Efficiency")

# axs.set_xlim(efficiencies["# mag"].min(), efficiencies["# mag"].max())
axs.set_xlim(17, efficiencies["# mag"].max())
axs.set_ylim(0, None)

plt.savefig("efficiencies.pdf")
plt.close()

# SM: note that if `mag_r` is used instead of `truth_mag_r`, then some of the
#     bins will have values above 1 due to inaccurate mag measurements

# Maglim Map

In [23]:
maglim_map = hp.read_map('/home/smau/WORK/satsim/supreme_dc2_dr6d_v3_r_maglim_psf_wmean.fits.gz')

In [24]:
maglim = maglim_map[hp.ang2pix(4096, coord.ra.value, coord.dec.value, lonlat=True)]

In [25]:
print(maglim)

26.791002


In [26]:
# hp.mollview(maglim_map)

In [27]:
_nbins = 100
_bins_x = np.linspace(16, 28, _nbins + 1)
_bins_y = np.logspace(-4.1, 0.2, _nbins + 1)
H, mag_bins, mag_err_bins, _ = stats.binned_statistic_2d(
    true_stars['truth_mag_r'],
    true_stars['mag_err_r'],
    None,
    statistic="count",
    bins=[_bins_x, _bins_y],
)

In [28]:
mag_err_r, _, _ = stats.binned_statistic(
    true_stars['truth_mag_r'],
    true_stars['mag_err_r'],
    statistic="median",
    bins=mag_bins,
)
mag_centers = (mag_bins[:-1] + mag_bins[1:]) / 2

In [29]:
# print("delta_mag, log mag_err_r")
# out = np.array([mag_centers - maglim, np.log10(mag_err_r)]).T
# print(out)
# np.savetxt('phot.csv', out, delimiter=',', header='delta_mag,log_mag_err')

# Photometric Error Plot

In [30]:
photometric_err = pd.read_csv("phot.csv")

# we are using `truth_mag_r` because we're using these photometric error models
# to generate simulations which will then be used to generate realistic photometry

fig, axs = plt.subplots(
    1, 1,
    figsize=new_figsize,
    subplotpars=new_subplotpars,
)

axs.pcolormesh(
    mag_bins,
    mag_err_bins,
    H.T,
    norm="log",
    cmap='binary',
    rasterized=True,
)
# axs.plot(
#     mag_centers,
#     mag_err_r,
#     c='r',
#     ls=":",
#     label="DP0.2",
# )
axs.plot(
    photometric_err["# delta_mag"] + maglim,
    10**photometric_err["log_mag_err"],
    c='r',
    label="DP0.1",
)
axs.axvline(
    maglim,
    ls=':',
    color='k',
)
# axs.axhline(
#     0.1085,
#     ls='--',
#     color='k',
# )

axs.set_yscale("log")
axs.set_xlabel('$r$ [mag]')
axs.set_ylabel('$\sigma_r$ [mag]')


def get_delta_mag(mag):
    return mag - maglim

def get_abs_mag(mag):
    return mag + maglim

secax = axs.secondary_xaxis("top", functions=(get_delta_mag, get_abs_mag))
secax.set_xlabel('$\Delta r$ [mag]')
# plt.legend()

plt.savefig("photometric_err.pdf")
plt.close()

---

In [31]:
print(figheight)

2.5453124999999996


In [32]:
figwidth = 7.1
# figheight = 4.8 / 6.4 * figwidth  # 2.5453124999999996
figsize = (figwidth, new_figheight)
print(figsize)

(7.1, 3.022558593749999)


In [33]:
new_subplotpars = mpl.gridspec.SubplotParams(
    left=0.1,
    right=0.95,
    bottom=new_bottom,
    top=new_top,
    wspace=0.3,
    hspace=0.2
)

In [40]:

def get_delta_mag(mag):
    return mag - maglim

def get_abs_mag(mag):
    return mag + maglim

fig, axs = plt.subplots(
    1, 2,
    figsize=figsize,
    subplotpars=new_subplotpars,
    sharex=True,
)

axs[0].axvline(
    maglim,
    ls=':',
    color='gray',
    # lw=2,
)
axs[0].plot(
    efficiencies["# mag"],
    efficiencies["detection_eff"],
    label='Detection',
    ls="-",
    c="gray",
    # lw=2,
)  # matched objects with truth class of stars / true stars
axs[0].plot(
    efficiencies["# mag"],
    efficiencies["classifiction_eff"],
    label='Classification',
    ls="-",
    c="k",
    # lw=2,
)  # matched objects correctly classified as stars / matched objects with truth class of stars
axs[0].plot(
    efficiencies["# mag"],
    efficiencies["classification_detection_eff"],
    label='Detection &\nClassification',
    ls="-",
    c="r",
    # lw=2,
)  # matched objects correctly classified as stars / true stars

secax = axs[0].secondary_xaxis("top", functions=(get_delta_mag, get_abs_mag))
secax.set_xlabel('$\Delta r$ [mag]')

axs[0].legend(loc="lower left")

axs[0].set_xlabel("$r$ [mag]")
axs[0].set_ylabel("Efficiency")

# axs[0].set_xlim(efficiencies["# mag"].min(), efficiencies["# mag"].max())
# axs[0].set_xlim(16, efficiencies["# mag"].max())
# axs[0].set_xlim(16, 28)
axs[0].set_ylim(0, None)


axs[1].pcolormesh(
    mag_bins,
    mag_err_bins,
    H.T,
    norm="log",
    cmap='binary',
    rasterized=True,
)
# axs.plot(
#     mag_centers,
#     mag_err_r,
#     c='r',
#     ls=":",
#     label="DP0.2",
# )
axs[1].plot(
    photometric_err["# delta_mag"] + maglim,
    10**photometric_err["log_mag_err"],
    c='r',
    label="Median Photometric Error",
    # lw=2,
)
axs[1].axvline(
    maglim,
    ls=':',
    color='gray',
    # lw=2,
)
# axs.axhline(
#     0.1085,
#     ls='--',
#     color='k',
# )
axs[1].legend(loc="upper left")

axs[1].set_yscale("log")
axs[1].set_xlabel('$r$ [mag]')
axs[1].set_ylabel('$\sigma_r$ [mag]')

# xmin = (photometric_err["# delta_mag"] + maglim).min()
# xmax = (photometric_err["# delta_mag"] + maglim).max()
xmin = 17
xmax = 28
axs[1].set_xlim(xmin, xmax)

secax = axs[1].secondary_xaxis("top", functions=(get_delta_mag, get_abs_mag))
secax.set_xlabel('$\Delta r$ [mag]')
# plt.legend()

plt.savefig("efficiency-photerr.pdf")
plt.close()